In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [ ]:
df_users = sc.textFile('./moviedata/users.csv')
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

In [ ]:
zipcode_user = df_users.map(lambda x: x.split(",")).map(lambda x:  (str(x[4])[1:-1],str(x[0])))
zipcode_user.take(5)

In [ ]:
zipcode_state = df_zipcodes.map(lambda x: x.split(",")).map(lambda x:  (str(x[0])[1:-1], str(x[3])[1:-1]))
zipcode_state.take(5)

In [ ]:
zipcode_user_state = zipcode_user.join(zipcode_state)
zipcode_user_state.take(5)

In [ ]:
user_state = zipcode_user_state.map(lambda x: x[1])
user_state.take(5)

In [ ]:
user_movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x: (str(x[0]), (str(x[1]),int(x[2]))))
user_movieid_rating.take(5)

In [ ]:
user_movieid_rating_state = user_movieid_rating.join(user_state)
user_movieid_rating_state.take(5)

In [ ]:
movieid_state_rating_count = user_movieid_rating_state.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))
movieid_state_rating_count.take(5)

In [ ]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + y[2].split(',')[2:]

In [ ]:
df_movies_cleaned = df_movies.map(lambda x : clean_df_movies(x))

In [ ]:
i = 2
movieid_genre = df_movies_cleaned.filter(lambda x: int(x[i]) == 1).map(lambda x: (str(x[0]), i))
movieid_genre.take(5)

In [ ]:
#New
#Pseudocode:
#for each genre:
#  get list of movies in the above genre
#  join list of movies with rating
#  joining this with users to get zipcodes
#  join this with zipcodes to get state
#  reduce on basis of states to get average rating
#  return the best average rating state(requirement of question)


In [ ]:
df_users = sc.textFile('./moviedata/users.csv').map(lambda x: x.split(","))
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv').map(lambda x: x.split(","))
df_rating = sc.textFile('./moviedata/rating.csv').map(lambda x: x.split(","))
df_movies = sc.textFile('./moviedata/movies.csv')

In [ ]:
movieid_userid_rating = df_rating.map(lambda x: (x[1] , (x[0], float(x[2]))))

In [ ]:
userid_zipcode = df_users.map(lambda x : (x[0], x[4]))

In [ ]:
zipcode_state = df_zipcodes.map(lambda x: (x[0], x[3]))

In [ ]:
movieid_userid_rating.take(5)

In [ ]:
#Clean df_movies
df_movies_cleaned = df_movies.map(lambda x: clean_df_movies(x))

In [ ]:
for genre_id in range(1, 20):
    #  get list of movies in the above genre
    movie_list = df_movies_cleaned.filter(lambda x : int(x[genre_id]) == 1).map(lambda x : (x[0], genre_id))

    #  join list of movies with rating
    userid_rating_list = movie_list.join(movieid_userid_rating).map(lambda x : (x[1][1][0], x[1][1][1]))

    #  joining this with users to get zipcodes
    zipcode_rating_list = userid_rating_list.join(userid_zipcode).map(lambda x : (x[1][1], x[1][0]))

    #  join this with zipcodes to get state
    state_rating_list = zipcode_rating_list.join(zipcode_state).map(lambda x : (x[1][1], (x[1][0], 1)))

    #  reduce on basis of states to get average rating
    state_avg_rating_tuple = state_rating_list.reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))
    state_avg_rating = state_avg_rating_tuple.map(lambda x: (x[0], x[1][0]/x[1][1]))
    
    

In [ ]:
from pyspark.statcounter import StatCounter
state_avg_rating.aggregate(StatCounter(), StatCounter.merge, StatCounter.mergeStats).maxValue

In [ ]:
movie_user_id_rating_list.take(10)